This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [117]:
# First, import the relevant modules
import requests
from statistics import mean

In [136]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#Set the API url to the one corresponding to Carl Zeiss. Set Filter in the URL  to retrieve data for Jan 5th, 2017 (Thursday) to examine the data
api_url_weekday = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-05&end_date=2017-01-05&api_key="
session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data_weekday = session.get(api_url_weekday)
afx_stock_weekday=raw_data_weekday.json()
print(afx_stock_weekday)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-05-06T22:31:14.486Z', 'newest_available_date': '2019-05-06', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-05', 'end_date': '2017-01-05', 'data': [['2017-01-05', 35.02, 35.2, 34.73, 35.06, None, 48412.0, 1692326.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


In [138]:
# Now let us examine the data on weekend when stock market is closed. Jan 7,2017 was Saturday.
# We notice that the entire data column is empty
api_url_weekend = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-07&end_date=2017-01-07&api_key="
session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data_weekend = session.get(api_url_weekend)
afx_stock_weekend=raw_data_weekend.json()
print(afx_stock_weekend)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-05-06T22:31:14.486Z', 'newest_available_date': '2019-05-06', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-07', 'end_date': '2017-01-07', 'data': [], 'collapse': None, 'order': None, 'database_id': 6129}}


In [145]:
# Retrieve data for whole of 2017
api_url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key="
session = requests.Session()
session.mount('http://', requests.adapters.HTTPAdapter(max_retries=3))
raw_data = session.get(api_url)

# Return json data as dict
afx_stock = raw_data.json()

# Extract a list of lists containing stock data for all the entries for this year
data_list=afx_stock['dataset']['data']

# Extract a list containing the entries under the column_names (such as 'Date','Open','High' etc.)
column_list=afx_stock['dataset']['column_names']

# Create a list of dictionaries by zipping column_list with data_list so we have the column names as keys and data as values
zip_column_data=[zip(item_list,data_list_item) for data_list_item in data_list]
stock_info=[]
for item in zip_column_data:
    stock_info.append(dict(item))
    
# Define a function which takes a list as argument and returns only non-null values (To take care of None values 
# corresponding to weekends)
def non_null_list(input_list):
    output_list=[i for i in input_list if i]
    return output_list

#Creating list of the opening price of the day from stock_info list of dicts
open_list=non_null_list([dict_item['Open'] for dict_item in stock_info])

#Extract maximum and minimum of opening price for whole year
max_open_price=max(open_list)
min_open_price=min(open_list)

# Create two lists which contain all the highs and lows of the stock respectively
high_list=non_null_list([dict_item['High'] for dict_item in stock_info])
low_list = non_null_list([dict_item['Low'] for dict_item in stock_info])

# Create a list which stores the difference of the high price and the low price and calculate the maximum intra-day price change
price_change_list=[x-y for x,y in zip(high_list,low_list)]
max_price_change=round(max(price_change_list),2)

# Create a list with closing price and create another list with closing price difference for consecutive days
close_list=non_null_list([dict_item['Close'] for dict_item in stock_info])
close_change_list=[close_list[i+1]-close_list[i] for i in range(len(close_list)-1)] #list of closing price diff for consecutive days

# Printing max difference between closing price for consecutive days
max_close_price_diff=round(max(close_change_list),2)

# Create list of the trading volume for each day and calculate the average traded volume for the whole year
trading_vol_list=non_null_list([dict_item['Traded Volume'] for dict_item in stock_info])
average_traded_vol=round(mean(trading_vol_list),2)

# Define a function which returns median from a list of numbers
def median_calc(list_vol):
    list_vol.sort()
    len_list=len(list_vol)
    if len_list%2==0:
        median_list=((list_vol[int(len_list/2)]+list_vol[int((len_list)/2-1)]))/2
    else:
        median_list=list_vol[int(len_list/2)]
    return int(median_list)

# Calculate median using the function defined above
median_trading_vol=median_calc(trading_vol_list)  

## Print out the required quantities ##

print("The highest opening price for the stock in 2017 was $",max_open_price)
print("The lowest opening price for the stock in 2017 was $",min_open_price)
print("THe largest intra-day range for the stock in any one day of 2017 was $", max_price_change)
print("The largest difference between closing price for consecutive days in 2017 was $",max_close_price_diff)
print("The average daily trading volume for the stock in 2017 was", average_traded_vol)
print("The median trading volume for the stock in 2017 was",median_trading_vol )


The highest opening price for the stock in 2017 was $ 53.11
The lowest opening price for the stock in 2017 was $ 34.0
THe largest intra-day range for the stock in any one day of 2017 was $ 2.81
The largest difference between closing price for consecutive days in 2017 was $ 2.56
The average daily trading volume for the stock in 2017 was 89124.34
The median trading volume for the stock in 2017 was 76286


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)